# base

> Base classes, types, and protocols for Tailwind CSS abstractions

In [ ]:
#| default_exp core.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Union, Optional, Literal, Protocol, runtime_checkable, TypeVar, Generic, Callable, Any, Dict, List, Tuple
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum
import re

## Core Types

Define the fundamental types used throughout the library:

In [ ]:
#| export
# Type aliases for Tailwind values
TailwindScale = Union[int, float, str]  # Numeric scales like 1, 2.5, "px"
TailwindFraction = str  # Fractions like "1/2", "2/3"
TailwindArbitrary = str  # Arbitrary values like "123px", "10rem"
TailwindCustomProperty = str  # CSS custom properties like "--spacing-lg"
TailwindValue = Union[TailwindScale, TailwindFraction, TailwindArbitrary, TailwindCustomProperty] # Union of all possible value types

## Value Validators

Functions to validate and identify different types of Tailwind values:

In [ ]:
#| export
def is_numeric_scale(
    value: Any  # The value to check - can be int, float, or string
) -> bool:  # True if the value is a valid numeric scale, False otherwise
    """Check if value is a valid numeric scale (int, float, or 'px')."""
    if isinstance(value, (int, float)):
        return True
    if isinstance(value, str) and value == "px":
        return True
    return False

In [ ]:
#| export
def is_fraction(
    value: Any  # The value to check for fraction format
) -> bool:  # True if the value is a valid fraction string, False otherwise
    """Check if value is a valid fraction string (e.g., '1/2', '3/4')."""
    if not isinstance(value, str):
        return False
    pattern = r'^\d+/\d+$'
    return bool(re.match(pattern, value))

In [ ]:
#| export
def is_custom_property(
    value: Any  # The value to check for CSS custom property format
) -> bool:  # True if the value is a CSS custom property, False otherwise
    """Check if value is a CSS custom property (starts with --)."""
    if not isinstance(value, str):
        return False
    return value.startswith("--")

In [ ]:
#| export
def is_arbitrary_value(
    value: Any  # The value to check for arbitrary CSS value format
) -> bool:  # True if the value contains CSS units or calc(), False otherwise
    """Check if value is an arbitrary value (contains units or special chars)."""
    if not isinstance(value, str):
        return False
    # Check for common CSS units
    units = r'(px|em|rem|vh|vw|%|deg|s|ms)$'
    return bool(re.search(units, value)) or value.startswith("calc(")

## Base Protocol

Define the protocol that all utility builders must implement:

In [ ]:
#| export
@runtime_checkable
class TailwindBuilder(Protocol):
    """Protocol for all Tailwind utility builders."""
    
    def build(
        self,
        *args,
        **kwargs
    ) -> str:  # The built CSS class string
        """Build and return the CSS class string."""
        ...
    
    def __str__(
        self
    ) -> str:  # The built CSS class string
        """Return the built CSS class string."""
        ...

## Base Classes

Abstract base classes for different types of utility builders:

In [ ]:
#| export
class BaseUtility(ABC):
    """Base class for all Tailwind utility builders."""
    
    def __init__(
        self,
        prefix: str  # The utility prefix (e.g., 'w' for width, 'p' for padding)
    ):
        """
        Initialize with a utility prefix.
        """
        self.prefix = prefix
        self._value: Optional[str] = None
        self._modifiers: List[str] = []
    
    @abstractmethod
    def _format_value(
        self,
        value: TailwindValue  # The value to format (numeric, fraction, arbitrary, etc.)
    ) -> str:  # The formatted value string ready for class construction
        """Format the value according to Tailwind conventions."""
        pass
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use for building the class
    ) -> str:  # The complete CSS class string with modifiers applied
        """Build the complete CSS class string."""
        if value is not None:
            formatted_value = self._format_value(value)
        elif self._value is not None:
            formatted_value = self._value
        else:
            return self.prefix
        
        # Join prefix and value
        base_class = f"{self.prefix}-{formatted_value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class
    
    def build(
        self,
        value: Optional[TailwindValue] = None  # Optional value to override the stored value
    ) -> str:  # The built CSS class string
        """Build and return the CSS class string."""
        return self._build_class(value)
    
    def __str__(
        self
    ) -> str:  # The built CSS class string
        """Return the built CSS class string."""
        return self._build_class()

## Standard Value Formatter

A standard implementation for formatting Tailwind values:

In [ ]:
#| export
class StandardUtility(BaseUtility):
    """Standard utility class with common value formatting."""
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format according to Tailwind conventions
    ) -> str:  # The formatted value string (e.g., "4", "[10px]", "(--custom)")
        """
        Format value according to Tailwind conventions:
        - Numeric scales: used as-is (e.g., 4 -> "4")
        - Fractions: used as-is (e.g., "1/2" -> "1/2")
        - Custom properties: wrapped in parentheses (e.g., "--spacing" -> "(--spacing)")
        - Arbitrary values: wrapped in brackets (e.g., "10px" -> "[10px]")
        """
        if is_numeric_scale(value):
            return str(value)
        elif is_fraction(value):
            return value
        elif is_custom_property(value):
            return f"({value})"
        elif is_arbitrary_value(value):
            return f"[{value}]"
        else:
            # Named values (like 'auto', 'full', etc.)
            return str(value)

## Named Scale Utilities

For utilities that support named scales (e.g., sm, md, lg):

In [ ]:
#| export
@dataclass
class NamedScale:
    """Represents a named scale with optional CSS variable."""
    name: str
    var: Optional[str] = None
    comment: Optional[str] = None
    
    def format(
        self
    ) -> str:  # The name of the scale for use in CSS classes
        """Format as Tailwind class suffix."""
        return self.name

In [ ]:
#| export
CONTAINER_SCALES = [ # Common named scales used across utilities
    NamedScale("3xs", "--container-3xs", "16rem (256px)"),
    NamedScale("2xs", "--container-2xs", "18rem (288px)"),
    NamedScale("xs", "--container-xs", "20rem (320px)"),
    NamedScale("sm", "--container-sm", "24rem (384px)"),
    NamedScale("md", "--container-md", "28rem (448px)"),
    NamedScale("lg", "--container-lg", "32rem (512px)"),
    NamedScale("xl", "--container-xl", "36rem (576px)"),
    NamedScale("2xl", "--container-2xl", "42rem (672px)"),
    NamedScale("3xl", "--container-3xl", "48rem (768px)"),
    NamedScale("4xl", "--container-4xl", "56rem (896px)"),
    NamedScale("5xl", "--container-5xl", "64rem (1024px)"),
    NamedScale("6xl", "--container-6xl", "72rem (1152px)"),
    NamedScale("7xl", "--container-7xl", "80rem (1280px)"),
]

## Modifier Support

Support for responsive and state modifiers:

In [ ]:
#| export
@dataclass
class Breakpoint:
    """Responsive breakpoint definition."""
    name: str
    min_width: Optional[str] = None

In [ ]:
#| export
BREAKPOINTS = { # Common breakpoints
    "sm": Breakpoint("sm", "640px"),
    "md": Breakpoint("md", "768px"),
    "lg": Breakpoint("lg", "1024px"),
    "xl": Breakpoint("xl", "1280px"),
    "2xl": Breakpoint("2xl", "1536px"),
}

In [ ]:
#| export
STATE_MODIFIERS = [ # Common state modifiers
    "hover", "focus", "active", "visited", "target",
    "focus-within", "focus-visible", "disabled", "enabled",
    "checked", "indeterminate", "default", "required",
    "valid", "invalid", "in-range", "out-of-range",
    "placeholder-shown", "autofill", "read-only",
    "first", "last", "odd", "even", "first-of-type",
    "last-of-type", "only-child", "empty",
    "before", "after", "first-letter", "first-line",
    "marker", "selection", "file", "backdrop",
    "placeholder", "open", "closed",
]

## Utility Factory

A factory function to create utility instances with method chaining:

In [ ]:
#| export
T = TypeVar('T', bound=BaseUtility)

In [ ]:
#| export
class UtilityFactory(Generic[T]):
    """Factory for creating utility instances with fluent API."""
    
    def __init__(
        self,
        utility_class: type[T],  # The utility class to instantiate
        prefix: str  # The prefix to use for the utilities
    ):
        "Initialize factory with a utility class and prefix."
        self.utility_class = utility_class
        self.prefix = prefix
    
    def __call__(
        self,
        value: Optional[TailwindValue] = None  # Initial value for the utility
    ) -> T:  # A new instance of the utility class
        """Create a utility instance with optional value."""
        instance = self.utility_class(self.prefix)
        if value is not None:
            instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(
        self,
        name: str  # Attribute name to convert to a utility value
    ) -> T:  # A new utility instance with the attribute as its value
        """Handle named values (e.g., w.full, h.screen)."""
        instance = self.utility_class(self.prefix)
        instance._value = name.replace("_", "-")
        return instance

## Examples

Let's test the base architecture with some examples:

In [ ]:
# Test numeric scale
assert is_numeric_scale(4) == True
assert is_numeric_scale(2.5) == True
assert is_numeric_scale("px") == True
assert is_numeric_scale("auto") == False

In [ ]:
# Test fraction detection
assert is_fraction("1/2") == True
assert is_fraction("3/4") == True
assert is_fraction("1.5") == False
assert is_fraction("full") == False

In [ ]:
# Test custom property detection
assert is_custom_property("--spacing-lg") == True
assert is_custom_property("--color-primary") == True
assert is_custom_property("spacing-lg") == False

In [ ]:
# Test arbitrary value detection
assert is_arbitrary_value("10px") == True
assert is_arbitrary_value("2.5rem") == True
assert is_arbitrary_value("100%") == True
assert is_arbitrary_value("calc(100% - 20px)") == True
assert is_arbitrary_value("auto") == False

In [ ]:
# Test StandardUtility class
class TestUtility(StandardUtility):
    pass

# Create test instances
util = TestUtility("w")

# Test different value types
assert util.build(4) == "w-4"
assert util.build("1/2") == "w-1/2"
assert util.build("--custom") == "w-(--custom)"
assert util.build("10px") == "w-[10px]"
assert util.build("auto") == "w-auto"

In [ ]:
# Test UtilityFactory
w = UtilityFactory(TestUtility, "w")

# Test factory patterns
assert str(w(4)) == "w-4"
assert str(w.full) == "w-full"
assert str(w.auto) == "w-auto"
assert str(w.screen) == "w-screen"

## Class Combination Utility

A utility function to combine multiple class builders:

In [ ]:
#| export
def combine_classes(
    *args: Union[str, BaseUtility, TailwindBuilder, None]
) -> str:  # Space-separated class string
    "Combine multiple class builders or strings into a single class string."
    classes = []
    for arg in args:
        if arg is None:
            continue
        elif isinstance(arg, str):
            if arg.strip():  # Only add non-empty strings
                classes.append(arg.strip())
        elif isinstance(arg, BaseUtility):
            classes.append(str(arg))
        elif hasattr(arg, 'build'):
            classes.append(arg.build())
        elif hasattr(arg, '__str__'):
            result = str(arg)
            if result.strip():
                classes.append(result)
    
    return " ".join(classes)

In [ ]:
# Test combine_classes
w_factory = UtilityFactory(TestUtility, "w")
h_factory = UtilityFactory(TestUtility, "h")

result = combine_classes(
    w_factory(4),
    h_factory.full,
    "flex",
    None,
    "items-center"
)
assert result == "w-4 h-full flex items-center"

## Directional Utilities

Support for utilities with directional variants (top, right, bottom, left):

In [ ]:
#| export
@dataclass
class Direction:
    """Represents a directional variant."""
    suffix: str
    css_suffix: str

In [ ]:
#| export
DIRECTIONS = { # Common directions
    "t": Direction("t", "top"),      # top
    "r": Direction("r", "right"),    # right
    "b": Direction("b", "bottom"),   # bottom
    "l": Direction("l", "left"),     # left
    "x": Direction("x", "inline"),   # horizontal
    "y": Direction("y", "block"),    # vertical
}

In [ ]:
#| export
class DirectionalUtility(StandardUtility):
    """Base class for utilities with directional variants."""
    
    def __init__(
        self,
        prefix: str,  # Base prefix (e.g., 'p' for padding)
        direction: Optional[str] = None  # Optional direction ('t', 'r', 'b', 'l', 'x', 'y')
    ):
        """Initialize with prefix and optional direction."""
        if direction and direction in DIRECTIONS:
            full_prefix = f"{prefix}{direction}"
        else:
            full_prefix = prefix
        super().__init__(full_prefix)

In [ ]:
# Test directional utilities
pt = DirectionalUtility("p", "t")
assert pt.build(4) == "pt-4"

px = DirectionalUtility("p", "x")
assert px.build(8) == "px-8"

## Negative Value Support

Support for utilities that can have negative values:

In [ ]:
#| export
class NegativeableUtility(StandardUtility):
    """Utility class that supports negative values."""
    
    def __init__(
        self,
        prefix: str,  # Base prefix
        negative: bool = False  # Whether this is a negative variant
    ):
        """Initialize with prefix and negative flag."""
        self.negative = negative
        full_prefix = f"-{prefix}" if negative else prefix
        super().__init__(full_prefix)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()